# Chapter 19: Transactions and MVCC

PostgreSQL's Multi-Version Concurrency Control (MVCC) provides high concurrency without excessive locking, but introduces complexity around visibility, isolation, and resource management. This chapter explains how PostgreSQL handles concurrent data modifications, the practical implications of isolation levels, and patterns for maintaining correctness under load.

## 19.1 MVCC Architecture

Unlike databases that use read locks, PostgreSQL maintains multiple versions of rows to allow readers and writers to proceed without blocking each other.

### 19.1.1 Row Versioning and System Columns

```sql
-- Every PostgreSQL table has hidden system columns:
-- xmin: Transaction ID that inserted this row version
-- xmax: Transaction ID that deleted/updated this row (0 = active)
-- cmin/cmax: Command ID within transaction
-- ctid: Physical location (block, offset) - changes after VACUUM FULL

-- Viewing row versions (requires superuser or specific permissions):
SELECT 
    xmin, 
    xmax, 
    ctid, 
    user_id, 
    email, 
    status 
FROM users 
WHERE user_id = 123;

-- Example output:
-- xmin   | xmax | ctid   | user_id | email      | status
-- 452001 | 0    | (0,15) | 123     | alice@...  | active
-- 452001 = Inserted by transaction 452001
-- 0      = Not deleted (active version)
-- (0,15) = Block 0, offset 15

-- When a row is updated, PostgreSQL creates a new version:
BEGIN;
UPDATE users SET status = 'inactive' WHERE user_id = 123;
-- Creates new row version with:
--   xmin = current_transaction_id (e.g., 452100)
--   xmax = 0
-- Old version updated:
--   xmax = 452100 (marking it deleted by transaction 452100)
COMMIT;

-- Now querying:
SELECT xmin, xmax, ctid, status FROM users WHERE user_id = 123;
-- New version: xmin=452100, xmax=0, ctid=(0,16), status='inactive'
-- Old version still exists on disk: xmin=452001, xmax=452100
-- Old version is "dead" but remains until VACUUM removes it
```

### 19.1.2 The Visibility Check

When a transaction queries data, PostgreSQL applies visibility rules to determine which row versions are visible:

```sql
-- Visibility logic (simplified):
-- 1. Row is visible if:
--    a) xmin is committed AND xmin < transaction's snapshot xid
--    b) xmax is 0 OR xmax is aborted OR xmax > transaction's snapshot xid
-- 2. Row is invisible (deleted) if:
--    a) xmax is committed AND xmax < transaction's snapshot xid

-- Practical demonstration:
-- Session A (Transaction 1):
BEGIN;
SELECT txid_current();  -- Returns 100
UPDATE accounts SET balance = 900 WHERE id = 1;  -- Was 1000
-- Does not commit yet

-- Session B (Transaction 2):
BEGIN;
SELECT txid_current();  -- Returns 101
SELECT balance FROM accounts WHERE id = 1;  -- Sees 1000 (old version)
-- Why? Transaction 100 is not committed yet, so its changes are invisible
-- Session B sees the version where xmax=0 or xmax > 101

-- Session A commits:
COMMIT;  -- Transaction 100 commits

-- Session B (still in transaction 101):
SELECT balance FROM accounts WHERE id = 1;  -- Still sees 1000!
-- Why? Session B's snapshot was taken at BEGIN, showing committed transactions as of that moment
-- Transaction 100 committed after 101 started, but 101 doesn't see it yet

-- Session B commits and starts new transaction:
COMMIT;
BEGIN;
SELECT balance FROM accounts WHERE id = 1;  -- Now sees 900
```

## 19.2 Transaction Isolation Levels

SQL standard defines four isolation levels. PostgreSQL implements three (Read Uncommitted behaves like Read Committed).

### 19.2.1 Read Committed (Default)

```sql
-- Default isolation level
-- Behavior:
-- 1. Each statement sees a fresh snapshot (committed data as of statement start)
-- 2. Within a transaction, subsequent statements may see newer committed data
-- 3. UPDATE/DELETE only see target rows committed as of statement start

-- Demonstration of statement-level consistency:
BEGIN;
-- Statement 1:
SELECT COUNT(*) FROM orders WHERE status = 'pending';  -- Returns 50
-- Meanwhile, another session commits 10 new pending orders
-- Statement 2 (same transaction):
SELECT COUNT(*) FROM orders WHERE status = 'pending';  -- Returns 60!
-- Different statements see different snapshots

-- Demonstration of concurrent update behavior:
-- Session A:
BEGIN;
UPDATE accounts SET balance = balance - 100 WHERE id = 1;  -- Balance was 1000
-- Now has lock on row id=1, balance pending 900

-- Session B (concurrent):
BEGIN;
UPDATE accounts SET balance = balance - 100 WHERE id = 1;  -- Blocked!
-- Waits for Session A's lock

-- Session A commits:
COMMIT;

-- Session B unblocks:
-- PostgreSQL re-evaluates WHERE clause with fresh snapshot
-- Sees balance is now 900 (Session A committed)
-- Applies update: 900 - 100 = 800
UPDATE returns "UPDATE 1" (success, not lost update)
COMMIT;
-- Final balance: 800 (correct)

-- However, Read Committed allows non-repeatable reads:
BEGIN;
SELECT balance FROM accounts WHERE id = 1;  -- Returns 800
-- Session B updates balance to 700 and commits
SELECT balance FROM accounts WHERE id = 1;  -- Returns 700 (different!)
COMMIT;
```

### 19.2.2 Repeatable Read

```sql
-- SET TRANSACTION ISOLATION LEVEL REPEATABLE READ;
-- Behavior:
-- 1. Single snapshot for entire transaction (taken at first statement)
-- 2. Subsequent statements see same data as first statement
-- 3. Prevents non-repeatable reads and phantom reads (in PostgreSQL)
-- 4. Serialization failures possible (must retry)

-- Preventing non-repeatable reads:
BEGIN ISOLATION LEVEL REPEATABLE READ;
SELECT balance FROM accounts WHERE id = 1;  -- Returns 800, snapshot taken
-- Session B updates to 700 and commits
SELECT balance FROM accounts WHERE id = 1;  -- Still returns 800 (same snapshot)
COMMIT;

-- Write skew demonstration (allowed in Repeatable Read):
-- Constraint: Sum of balances in checking + savings >= 0
-- Initial: Checking=500, Savings=500

-- Session A:
BEGIN ISOLATION LEVEL REPEATABLE READ;
SELECT balance FROM checking WHERE id = 1;  -- 500
SELECT balance FROM savings WHERE id = 1;   -- 500
-- Calculates: Can withdraw 600 from checking (500 + 500 - 600 >= 0)
UPDATE checking SET balance = -100 WHERE id = 1;
COMMIT;  -- Succeeds!

-- Session B (concurrent):
BEGIN ISOLATION LEVEL REPEATABLE READ;
SELECT balance FROM checking WHERE id = 1;  -- 500 (snapshot before A commits)
SELECT balance FROM savings WHERE id = 1;   -- 500
-- Calculates: Can withdraw 600 from savings (500 + 500 - 600 >= 0)
UPDATE savings SET balance = -100 WHERE id = 1;
COMMIT;  -- Succeeds!

-- Result: Checking=-100, Savings=-100, Sum=-200 (violated constraint!)
-- This is write skew - each transaction read data before the other wrote

-- Prevention: Use Serializable or explicit locking
```

### 19.2.3 Serializable

```sql
-- SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;
-- Behavior:
-- 1. Like Repeatable Read, but monitors for patterns that could cause anomalies
-- 2. If potential serialization anomaly detected, one transaction aborted with:
--    ERROR: could not serialize access due to read/write dependencies among transactions
-- 3. Application must retry aborted transactions

-- Same write skew example with Serializable:
BEGIN ISOLATION LEVEL SERIALIZABLE;
SELECT balance FROM checking WHERE id = 1;  -- 500
SELECT balance FROM savings WHERE id = 1;   -- 500
UPDATE checking SET balance = -100 WHERE id = 1;
COMMIT;  -- Session A succeeds

-- Session B:
BEGIN ISOLATION LEVEL SERIALIZABLE;
SELECT balance FROM checking WHERE id = 1;  -- 500 (snapshot)
SELECT balance FROM savings WHERE id = 1;   -- 500
UPDATE savings SET balance = -100 WHERE id = 1;
COMMIT;  
-- ERROR: could not serialize access due to read/write dependencies
-- DETAIL: Reason code: Canceled on identification as a pivot, during commit attempt
-- Session B must retry

-- Serializable uses predicate locking (less granular than row locking)
-- Higher overhead than Repeatable Read, but guarantees correctness

-- Practical usage:
-- Financial transfers where consistency across multiple rows is critical
-- Inventory allocation where sum constraints matter
-- Generally: Use explicit locking (SELECT FOR UPDATE) instead for better performance
-- Reserve Serializable for complex cases where predicate locking is easier than app logic
```

## 19.3 Write Skew and Anomaly Patterns

Understanding specific anomalies helps select appropriate mitigation strategies.

### 19.3.1 Lost Updates

```sql
-- Lost Update: Two transactions read same value, both update based on it, second overwrites first
-- Example: Counter increment

-- Session A:
BEGIN;
SELECT counter FROM stats WHERE id = 1;  -- Returns 100
-- Calculates: new value = 100 + 1

-- Session B:
BEGIN;
SELECT counter FROM stats WHERE id = 1;  -- Returns 100 (A not committed)
-- Calculates: new value = 100 + 1

-- Session A:
UPDATE stats SET counter = 101 WHERE id = 1;
COMMIT;

-- Session B:
UPDATE stats SET counter = 101 WHERE id = 1;  -- Overwrites A's update!
COMMIT;
-- Final value: 101 (should be 102)

-- Prevention in Read Committed:
-- UPDATE prevents lost updates by re-checking WHERE clause on lock acquisition
-- But only if UPDATE depends on previous read values in WHERE clause

-- Better prevention: Explicit row locking
BEGIN;
SELECT counter FROM stats WHERE id = 1 FOR UPDATE;  -- Locks row
-- Now others block until commit
UPDATE stats SET counter = counter + 1 WHERE id = 1;  -- Atomic increment (best)
COMMIT;

-- Or use advisory locks for complex logic
SELECT pg_advisory_lock(id) FROM stats WHERE id = 1;
-- Application logic, then update, then pg_advisory_unlock
```

### 19.3.2 Phantom Reads vs MVCC

```sql
-- Phantom Read: Query returns different set of rows on re-execution due to inserts
-- In PostgreSQL:
-- Read Committed: Phantoms possible between statements
-- Repeatable Read: Phantoms prevented for tuples, but not for aggregation conflicts

-- Example:
BEGIN ISOLATION LEVEL REPEATABLE READ;
SELECT SUM(amount) FROM transactions WHERE account_id = 1;  -- Sum = 1000
-- Session B inserts new transaction for account_id = 1 and commits
SELECT SUM(amount) FROM transactions WHERE account_id = 1;  -- Still 1000 (snapshot)
-- No phantom for tuples, but...

SELECT * FROM transactions WHERE account_id = 1;  -- Returns 5 rows (consistent with sum)
-- Insert by Session B is invisible (different xmax/xmin)

-- However, unique constraint violations can still occur:
-- Session A checks: SELECT 1 FROM accounts WHERE email = 'new@example.com' (no rows)
-- Session B inserts 'new@example.com' and commits
-- Session A tries: INSERT INTO accounts (email) VALUES ('new@example.com')
-- Result: Unique violation (not a phantom read, but a conflict)

-- True phantom prevention requires Serializable or explicit locking
BEGIN ISOLATION LEVEL SERIALIZABLE;
SELECT * FROM transactions WHERE account_id = 1;
-- Predicate lock established on account_id = 1
-- Session B insert blocked or causes serialization failure
```

## 19.4 Transaction Boundaries in Application Code

Proper transaction scoping is critical for both correctness and performance.

### 19.4.1 Autocommit vs Explicit Transactions

```sql
-- Autocommit mode (default in psql, most drivers):
-- Each statement is its own transaction
-- Implications:
-- 1. No multi-statement atomicity
-- 2. No rollback capability across operations
-- 3. Higher transaction overhead (commit per statement)

-- Bad pattern (pseudo-code):
for item in cart_items:
    db.execute("UPDATE inventory SET qty = qty - 1 WHERE sku = %s", item.sku)
    db.execute("INSERT INTO order_items (order_id, sku) VALUES (%s, %s)", order_id, item.sku)
-- If second statement fails, inventory update not rolled back (autocommit per statement)

-- Correct pattern: Explicit transaction
BEGIN;
UPDATE inventory SET qty = qty - 1 WHERE sku = 'ABC123';
INSERT INTO order_items (order_id, sku) VALUES (1001, 'ABC123');
COMMIT;
-- Atomic: both succeed or both fail

-- Application patterns by language:

-- Python (psycopg2/psycopg3):
-- with connection.transaction():
--     cursor.execute("UPDATE ...")
--     cursor.execute("INSERT ...")

-- Java (JDBC):
-- connection.setAutoCommit(false);
-- try {
--     stmt.executeUpdate("UPDATE ...");
--     stmt.executeUpdate("INSERT ...");
--     connection.commit();
-- } catch (SQLException e) {
--     connection.rollback();
-- }

-- Node.js (pg):
-- await client.query('BEGIN');
-- try {
--   await client.query('UPDATE ...');
--   await client.query('INSERT ...');
--   await client.query('COMMIT');
-- } catch (e) {
--   await client.query('ROLLBACK');
--   throw e;
-- }
```

### 19.4.2 Transaction Scope Optimization

```sql
-- Keep transactions as short as possible
-- Bad: Long transaction with user interaction
BEGIN;
SELECT * FROM products WHERE id = 1 FOR UPDATE;
-- Application shows product to user, waits for user input (30 seconds)
UPDATE products SET stock = stock - 1 WHERE id = 1;
COMMIT;
-- Problems:
-- 1. Row locked for 30 seconds (blocks other users)
-- 2. Connection held hostage
-- 3. Dead tuple accumulation (old version cannot be vacuumed)

-- Good: Optimistic approach
-- Application reads product (no lock)
-- User decides to buy
-- Optimistic update:
BEGIN;
UPDATE products SET stock = stock - 1 WHERE id = 1 AND stock > 0;
-- Check rows affected: If 0, out of stock (handled in app)
INSERT INTO orders ...
COMMIT;
-- Transaction duration: milliseconds

-- For complex business logic requiring multiple queries:
-- Use application-level state collection, then single transaction for writes
-- Or use advisory locks for long-duration logical locks without holding transaction
SELECT pg_advisory_xact_lock(resource_id);  -- Auto-released at transaction end
-- Or session-level:
SELECT pg_advisory_lock(resource_id);
-- ... long processing ...
SELECT pg_advisory_unlock(resource_id);
```

## 19.5 Locking Interactions with MVCC

MVCC eliminates read locks, but write locks and explicit locks remain necessary.

### 19.5.1 Row-Level Locks

```sql
-- FOR UPDATE: Exclusive lock, blocks other FOR UPDATE/SHARE
-- FOR SHARE: Shared lock, blocks FOR UPDATE but not other FOR SHARE
-- FOR NO KEY UPDATE: Like UPDATE (doesn't block FOR KEY SHARE)
-- FOR KEY SHARE: Like SELECT (blocks FOR UPDATE but not FOR NO KEY UPDATE)

-- Locking pattern to prevent write skew:
BEGIN ISOLATION LEVEL REPEATABLE READ;
SELECT * FROM accounts WHERE id IN (1, 2) FOR UPDATE;
-- Locks rows 1 and 2
-- Now calculation based on locked values is safe from concurrent modification
UPDATE accounts SET balance = balance - 100 WHERE id = 1;
UPDATE accounts SET balance = balance + 100 WHERE id = 2;
COMMIT;

-- FOR UPDATE SKIP LOCKED (PostgreSQL 9.5+):
-- Skip rows already locked by other transactions
-- Useful for queue processing:
BEGIN;
SELECT * FROM job_queue 
WHERE status = 'pending' 
ORDER BY priority DESC, created_at 
FOR UPDATE SKIP LOCKED 
LIMIT 1;
-- If another worker has locked the top job, this gets the next available
UPDATE job_queue SET status = 'processing', worker_id = 123 WHERE id = ?;
COMMIT;

-- NOWAIT: Fail immediately if cannot acquire lock
SELECT * FROM accounts WHERE id = 1 FOR UPDATE NOWAIT;
-- Raises ERROR: could not obtain lock on row in relation "accounts"
-- Application handles conflict immediately rather than waiting
```

### 19.5.2 Deadlocks

```sql
-- Deadlock: Circular lock dependency
-- Session A: Locks row 1, wants row 2
-- Session B: Locks row 2, wants row 1

-- Session A:
BEGIN;
UPDATE accounts SET balance = 900 WHERE id = 1;  -- Locks row 1
-- ... some logic ...
UPDATE accounts SET balance = 900 WHERE id = 2;  -- Waits for Session B

-- Session B:
BEGIN;
UPDATE accounts SET balance = 800 WHERE id = 2;  -- Locks row 2
-- ... some logic ...
UPDATE accounts SET balance = 800 WHERE id = 1;  -- Waits for Session A
-- DEADLOCK - PostgreSQL detects and aborts one transaction:
-- ERROR: deadlock detected
-- DETAIL: Process 12345 waits for ShareLock on transaction 67890; blocked by process 12346.
-- HINT: See server log for query details.

-- Prevention:
-- 1. Always acquire locks in consistent order (e.g., by ID ascending)
-- 2. Keep transactions short
-- 3. Use retry logic in application for deadlock errors

-- Application retry pattern (pseudo-code):
-- max_retries = 3
-- for attempt in 1..max_retries:
--     try:
--         db.execute("BEGIN")
--         process()
--         db.execute("COMMIT")
--         break
--     except DeadlockDetected:
--         db.execute("ROLLBACK")
--         if attempt == max_retries: raise
--         sleep(random(100ms, 500ms))  -- Exponential backoff
```

## 19.6 Long Transactions and Vacuum Impact

Long-running transactions are the primary cause of table bloat and performance degradation in PostgreSQL.

### 19.6.1 The xid Horizon Problem

```sql
-- PostgreSQL uses 32-bit transaction IDs (xid) that wrap around
-- XID exhaustion is prevented by "freezing" old rows (marking them as older than all current transactions)
-- Vacuum freezes rows older than vacuum_freeze_min_age

-- Long transaction blocks vacuum progress:
-- If a transaction started at xid 1000 is still running, 
-- rows with xmin < 1000 cannot be frozen (might still be visible to that transaction)
-- This prevents removal of dead tuples from before xid 1000

-- Check for long-running transactions:
SELECT 
    pid,
    usename,
    application_name,
    state,
    age(backend_xid) as xid_age,  -- How long transaction running
    now() - xact_start as duration,
    left(query, 100) as query
FROM pg_stat_activity
WHERE backend_xid IS NOT NULL  -- Active transactions only
  AND now() - xact_start > interval '5 minutes'
ORDER BY xact_start;

-- Check oldest xid preventing vacuum:
SELECT 
    relname,
    age(relfrozenxid) as xid_age,
    pg_size_pretty(pg_relation_size(oid)) as size
FROM pg_class
WHERE relkind = 'r'
ORDER BY age(relfrozenxid) DESC
LIMIT 10;
-- If age approaches 2 billion (wraparound limit), emergency vacuum required
-- autovacuum prioritizes tables near wraparound

-- Reckless query causing issues:
BEGIN;
SELECT * FROM huge_table;  -- Starts transaction, takes 2 hours to stream results
-- During this 2 hours:
-- 1. Dead tuples in huge_table cannot be vacuumed (table bloats)
-- 2. Autovacuum skips table (transaction might need old versions)
-- 3. Indexes bloat, queries slow down
COMMIT;
```

### 19.6.2 Idle in Transaction

```sql
-- "Idle in transaction" state: Transaction open but no active query
-- Usually caused by application connection pool or ORM keeping transactions open
-- Just as damaging as active long transactions for vacuum

-- Find idle-in-transaction connections:
SELECT 
    pid,
    usename,
    application_name,
    state,  -- 'idle in transaction'
    xact_start,
    now() - xact_start as idle_duration,
    left(query, 100) as last_query
FROM pg_stat_activity
WHERE state = 'idle in transaction'
  AND now() - xact_start > interval '10 minutes';

-- Prevention:
-- 1. Connection pool settings: Ensure connections returned to pool close transactions
-- 2. Application timeouts: Set statement_timeout and idle_in_transaction_session_timeout
ALTER SYSTEM SET idle_in_transaction_session_timeout = '60000';  -- 60 seconds
-- Automatically rolls back idle transactions after 60s

-- 3. Application discipline: Always commit/rollback before returning connection to pool
```

## 19.7 Savepoints and Subtransactions

Savepoints allow partial rollback within a transaction, but have overhead considerations.

```sql
-- Savepoint usage pattern:
BEGIN;
INSERT INTO orders (total) VALUES (100) RETURNING order_id;  -- Gets id=1
SAVEPOINT after_order;
INSERT INTO order_items (order_id, sku) VALUES (1, 'ABC');  -- Might fail (FK violation)
-- If fails:
ROLLBACK TO SAVEPOINT after_order;
-- Order remains, items rolled back
-- Can try alternative or continue
INSERT INTO order_items (order_id, sku) VALUES (1, 'DEF');  -- Success
COMMIT;

-- Overhead warning:
-- Each savepoint creates a subtransaction (subxid)
-- Excessive savepoints (hundreds per transaction) consume shared memory
-- and slow down transaction commit
-- Pattern to avoid:
-- BEGIN;
-- for item in items:
--     SAVEPOINT sp;
--     INSERT INTO ...;
--     RELEASE SAVEPOINT sp;  -- Or let it be released at commit
-- COMMIT;
-- If items = 1000, this creates 1000 subtransactions = performance disaster

-- Better: Batch inserts with exception handling in application, or use single transaction without savepoints for bulk operations
```

## 19.8 Industry Best Practices

### 19.8.1 Isolation Level Selection

```sql
-- Default (Read Committed): Use for most OLTP
-- - Good performance
-- - Statement-level consistency sufficient for most operations
-- - Handles concurrent updates safely (re-checks WHERE clause)

-- Repeatable Read: Use when:
-- - Generating reports requiring consistent view across multiple queries
-- - Complex reads requiring stable data during transaction
-- - Prevents non-repeatable reads but not all anomalies

-- Serializable: Use when:
-- - Critical financial calculations requiring absolute correctness
-- - Complex constraints spanning multiple tables (write skew prevention)
-- - Accept serialization failures and implement retry logic

-- Explicit Locking: Use when:
-- - Controlling access to specific resources (queues, counters)
-- - SELECT FOR UPDATE for pessimistic concurrency control
-- - Advisory locks for application-level synchronization

-- Pattern for high-contention counters:
-- Don't use SERIALIZABLE for simple increments (too expensive)
-- Instead:
UPDATE counters SET value = value + 1 WHERE id = 1 RETURNING value;
-- Atomic, no transaction isolation level needed for correctness
```

### 19.8.2 Transaction Size Management

```sql
-- Rule: Transactions should be as short as possible, as long as necessary

-- Bad: Gigantic transaction
BEGIN;
-- 100,000 inserts
INSERT INTO ...; -- x100000
COMMIT;
-- Issues:
-- 1. WAL generation huge (replication lag)
-- 2. Long hold on snapshot (blocks vacuum)
-- 3. Lock escalation risk
-- 4. Rollback takes forever if failure at end

-- Good: Batched transactions
-- Batch size: 1000-10000 rows depending on row width
-- Commit every batch
-- If failure, resume from last committed batch using idempotency keys

-- Idempotency pattern:
CREATE TABLE imports (
    batch_id UUID,
    record_number INT,
    data JSONB,
    processed_at TIMESTAMPTZ,
    PRIMARY KEY (batch_id, record_number)
);

-- Application:
-- for batch in chunks(data, 1000):
--     BEGIN;
--     INSERT INTO imports VALUES ... ON CONFLICT DO NOTHING;
--     COMMIT;
-- If retry, ON CONFLICT prevents duplicates
```

### 19.8.3 Monitoring Checklist

```sql
-- Daily transaction health check:
-- 1. Long-running transactions
SELECT count(*) FROM pg_stat_activity 
WHERE now() - xact_start > interval '1 hour';

-- 2. Idle in transaction
SELECT count(*) FROM pg_stat_activity 
WHERE state = 'idle in transaction';

-- 3. XID wraparound risk
SELECT max(age(relfrozenxid)) FROM pg_class WHERE relkind = 'r';

-- 4. Deadlock frequency (should be near zero)
-- Check PostgreSQL logs for deadlock messages
-- Metric: deadlocks per minute should be < 1

-- 5. Serialization failure rate (if using Serializable)
-- Check logs for "could not serialize access"
-- If high (>1% of transactions), consider lowering isolation or optimizing access patterns
```

---

## Chapter Summary

In this chapter, you learned:

1. **MVCC Architecture**: PostgreSQL maintains multiple row versions using `xmin` (inserting transaction) and `xmax` (deleting transaction) system columns. Readers never block writers; each transaction sees a snapshot of committed data as of statement or transaction start.

2. **Isolation Levels**: Read Committed (default) provides statement-level consistency with automatic re-evaluation of UPDATE WHERE clauses to prevent lost updates. Repeatable Read provides transaction-level snapshot consistency but allows write skew anomalies. Serializable prevents all anomalies including write skew through predicate locking but requires application retry logic for serialization failures.

3. **Write Skew**: Occurs when concurrent transactions read overlapping data sets, each makes decisions based on that data, and then write to disjoint rows, violating constraints. Prevent with explicit locking (`SELECT FOR UPDATE`), advisory locks, or Serializable isolation.

4. **Transaction Boundaries**: Keep transactions short; never hold transactions open during user interaction or network calls. Use explicit transactions (`BEGIN...COMMIT`) for multi-statement atomicity. Avoid "idle in transaction" states through connection pool configuration and `idle_in_transaction_session_timeout`.

5. **Locking**: Use row-level locks (`FOR UPDATE`, `FOR SHARE`) for pessimistic concurrency control. `SKIP LOCKED` enables efficient queue processing. Deadlocks occur with inconsistent lock ordering; applications must implement retry logic. Advisory locks provide application-level synchronization without database row contention.

6. **Long Transaction Impact**: Transactions prevent vacuum from removing dead tuples, causing table and index bloat. Monitor `pg_stat_activity` for `xid_age` and `idle in transaction` states. Tables near XID wraparound (2 billion limit) require urgent vacuuming; configure autovacuum aggressively for high-churn tables.

**Next:** In Chapter 20, we will explore Locking Deep Dive—covering explicit locking strategies, advisory locks, deadlock detection and resolution, and optimistic vs pessimistic concurrency control patterns for high-throughput systems.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../4. Indexing_and_query_performance/18. performance_tuning_playbook.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='20. isolation_levels_and_locking.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
